$\text{We are going to Use Python's package pyomo for solving the following assignment problem}$

UPak is a subsidiary of a transport company with LTL trucks.
UPak customers offer for each load a price in monetary units per cubic meter for the volume it occupies. Acceptance or not of the order to be transferred is left to UPak. The number of potential orders is a parameter N of the problem. The price offered by the customers for each load is given in the Profits [N] vector while the volume of the loads in cubic meters is given in the Size [N] vector
Customers bring their loads to the UPak terminal to be loaded on trailers whose capacity is finite. Note that the number of vehicles available to the company is a T parameter and the vehicle capacities in cubic meters are given in the Capacities [T] vector. Please note that partial loading of an order is not allowed. In other words, each load can be loaded at most in one vehicle.
UPak management wants to automate the process of selecting and assigning those orders to available vehicles in a way that maximizes its profits.
There are couples of orders that when placed in the same truck UPak receives some extra profit (this results from savings in the process of loading some orders). The above set of binary orders is given in the Combined vector [C], where C is the dimension of the vector. The CombinedProfits vector [C] contains the total profit for each such combination. For example, a unit of orders with indices 0 and 1 respectively can result in a profit of 120 and 93 km. per cubic meter, however, if and only if both are loaded on the same truck, UPak has an additional total profit from the combination, which is equal to 500 km.

$ \text{The formulation of the problem} $

$ \text{Variables :}  $

$ y_{ij} = \begin{cases} 1,& \text{if  i truck contains j order} \\ 0,& \text{if not}  
\end{cases} \\ \\ x_j =\begin{cases} 1,&\text{if  j order is going to be deliverd}\\ 0,& \text{ if not } \end{cases} \\ \\ \text{ i=0,1,2...,T-1 and j= 0,1,2,...,N-1  } \\ c_{ij} = \begin{cases} 1,& \text{If $D_i$ couple  is contained in truck j} \\0 ,& \text{if not} 
\end{cases}$ \\ 

$ \text {Objective: }  \\ ΜΑΧ \sum_{i=0}^{i=N-1}S_{i} P_{i} x_i + \sum_{i=0}^{i=C-1} \sum_{j=0}^{j=T-1}c_{ij}E_i $

$ \text{S.T :}  \\ x_j=\sum_{i=0}^{i=T-1} y_{ij} \ \  j=0,1,2,...,N-1 \\ \sum_{j=0}^{j=N-1} S_{j} y_{ij} \leq C_i \ i=0,1,2,....,T-1\\ $


$ \text{for every couple} \ (l,k)= D_0, D_1,..,D_{C-1} $  

$ c_{ij} \leq 1+2 -y_{jl}-y_{jk}  $


$ c_{ij} \geq 1-2 + y_{jl}+y_{jk} $

$ y_{jl}+y_{jk} \leq 2 + 2(1-c_{ij})$

$ y_{jl}+y_{jk} \geq 2- 2(1-c_{ij})\\ \\  ( \forall j : j \in          \{0,1,2,..,T-1\}) $

In [33]:
from pyomo.environ import *

import json


model=ConcreteModel()
with open('lowdimdata.json') as f:
    data = json.load(f)



Capacities=tuple(int(i) for i in data['Capacity'].split(','))
T=len(Capacities)

Profits=tuple(int(i) for i in data['Profit'].split(','))
N=len(Profits)


Sizes=tuple(int(i) for i in data['Size'].split(','))

C=len(data['Combined'])
Combined=tuple((tuple(data['Combined'][j].split(',')) for j in range(C)))

Combined=tuple(map(lambda x: (int(x[0]),int(x[1])) , Combined))

CombinedProfits=tuple(int(i ) for i in data['CombinedProfits'].split(','))

model.x=Var(range(N),domain=Binary)



model.y=Var(range(T),range(N) ,domain=Binary)
model.cp=Var(range(C),range(T),domain=Binary)


model.Cap=ConstraintList()
for j in range(T) :

    model.Cap.add(expr=sum(Sizes[i]*model.y[j,i] for i  in range(N)) <= Capacities[j])


def rule(model,i):
    return model.x[i]==sum(model.y[j,i] for j in range(T))

model.order_assign=Constraint(range(N),rule=rule )

model.combinedProf=ConstraintList()

for k in range(C):
    for j in range(T) :
        model.combinedProf.add(expr=model.cp[k,j]<=
                            1+ 2- model.y[j,Combined[k][0]]- model.y[j,Combined[k][1]])
        model.combinedProf.add(expr=model.cp[k,j]>=
                            1-2+ model.y[j,Combined[k][0]]+model.y[j,Combined[k][1]])

model.cpbo=ConstraintList()

for k in range(C):
    for j in range(T):
        model.cpbo.add(expr= model.y[j,Combined[k][0]]+model.y[j,Combined[k][1]] <= 2+2*(1-model.cp[k,j]))
        model.cpbo.add(expr= model.y[j,Combined[k][0]]+model.y[j,Combined[k][1]]>= 2-2*(1-model.cp[k,j]))
        
        


model.obj=Objective(
    expr= sum(Sizes[i]*Profits[i] *  model.x[i] for i in range(N))
                +sum(CombinedProfits[i]*sum(model.cp[i,j] for j in range(T)) for i in range(C) ),sense=maximize)


opt=SolverFactory('cplex')

opt.solve(model)


print('The Maximum profit is:  ',model.obj.value())
for i in range(T):
    s=[j for j in range(N) if model.y[i,j].value==1]
    
    print(f'orders assigned in truck {i} :    ' ,s)
    print(f'capacity used in truck {i}',sum(Sizes[j]*model.y[i,j].value for j in s))

    deprecated. Use the .expr property getter instead
The Maximum profit is:   8518.0
orders assigned in truck 0 :     [1, 7, 8]
capacity used in truck 0 35.0
orders assigned in truck 1 :     [4, 5, 6]
capacity used in truck 1 34.0


In [34]:
data

{'Capacity': '36,36',
 'Profit': '120,93,70,85,125,104,98,130,140,65',
 'Size': '5,11,22,15,7,9,18,14,10,12',
 'Combined': ['0,1', '5,6'],
 'CombinedProfits': '500,700'}